In [127]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
import cv2

import os
import sys
import subprocess as sb
import warnings
warnings.filterwarnings('ignore')

from rich import print as rprint
import torch
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 200)

import tensorflow as tf

from piq import brisque
import torchvision.transforms as T

In [2]:
import numpy as np
from IPython.display import Audio

# Generate a 440 Hz beep for 1 second
def play():
        
    sr = 44100  # sample rate
    duration = 1  # seconds
    t = np.linspace(0, duration, int(sr * duration), endpoint=False)
    wave = 0.5 * np.sin(2 * np.pi * 440 * t)
    
    return Audio(wave, rate=sr, autoplay=True)

In [3]:
pwd

'/home/prithwijit/programming/python/imp_projects/age_gender_project/notebooks'

In [ ]:
def age_interval(x):
    if x/5 <= 20:
        for i in range(20):
            if (i*5) <= x <= ((i+1)*5):
                return f'{i*5 + 1} to {(i+1)*5}'
            else:
                continue
    else:
        return '100+'

In [93]:
data_path = '../data/new_retail_data.csv'
df = pd.read_csv(data_path, usecols =['Transaction_ID', 'Name', 'Customer_ID', 'Country', 'Age', 'Gender', 'Income', 'Customer_Segment', 'Product_Category', 'products', 'Feedback', 'Ratings']).dropna()
df['age_group'] = df['Age'].apply(age_interval)
df['customer'] = df['Name'] + '__' + df['Customer_ID'].astype(int).astype(str) + '__' + df['Gender'].astype(str) + '__' + df['Country'] + '__' + df ['age_group']
df['invoice'] = df['Name'] + '__' + df['Customer_ID'].astype(int).astype(str) + df['products'].astype(str)
df.drop_duplicates(subset = ['invoice'], inplace = True)

In [131]:
df.Income.unique()

array(['Low', 'High', 'Medium'], dtype=object)

In [94]:
df.customer.nunique(), df.shape

(298037, (299064, 15))

In [99]:
user_item_matrix = df.query("Product_Category == 'Books'"].pivot_table(index = ['customer'], columns = 'Product_Category', values = 'Ratings', aggfunc = 'mean').fillna('0')
user_item_matrix.head()

Product_Category,Books,Clothing,Electronics,Grocery,Home Decor
customer,,,,,
Aaron Adams__29623__Male__USA__16 to 20,1.0,0,0,0,0
Aaron Adams__85537__Female__USA__16 to 20,0,0,2.0,0,0
Aaron Adams__97814__Female__Canada__46 to 50,0,0,5.0,0,0
Aaron Aguilar__21990__Female__Australia__41 to 45,0,0,0,4.0,0
Aaron Alexander__25057__Male__Canada__46 to 50,0,0,3.0,0,0


In [109]:
user_item_matrix = df.query("Product_Category == 'Books'").pivot_table(index = ['customer'], columns = 'products', values = 'Ratings', aggfunc = 'mean').fillna('0')
user_item_matrix.head()

products,Action,Adventure,Anthologies,Art supplies,Biography,Books,Building blocks,Business,Classic literature,Clothing,Contemporary literature,Cooking,Crime,Detective,Drama,Dystopian,Educational kits,Essays,Fantasy,Games,Health,Historical fiction,History,Horror,Legal thriller,Literary fiction,Memoir,Modern literature,Mystery,Non-fiction literature,Plush toys,Poetry,Political thriller,Psychological thriller,Psychology,Puzzles,Romance,Science,Science fiction,Self-help,Short stories,Sports equipment,Suspense,Techno-thriller,Thriller,Toys,Travel
customer,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Aaron Adams__29623__Male__USA__16 to 20,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Aaron Allen PhD__19707__Female__Germany__46 to 50,0,0,0,0,0,0,0,0,4.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Aaron Allen__40804__Male__UK__56 to 60,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2.0,0,0,0,0,0,0
Aaron Anderson__77727__Male__USA__46 to 50,0,0,0,0,0,4.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Aaron Anthony__89802__Female__Canada__21 to 25,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5.0,0,0,0,0,0,0,0


In [110]:
from sklearn.metrics.pairwise import cosine_similarity
item_sim = cosine_similarity(user_item_matrix.T)
item_sim_df = pd.DataFrame(
    item_sim,
    index=user_item_matrix.columns,
    columns=user_item_matrix.columns
)

In [114]:
item_sim_df

products,Action,Adventure,Anthologies,Art supplies,Biography,Books,Building blocks,Business,Classic literature,Clothing,Contemporary literature,Cooking,Crime,Detective,Drama,Dystopian,Educational kits,Essays,Fantasy,Games,Health,Historical fiction,History,Horror,Legal thriller,Literary fiction,Memoir,Modern literature,Mystery,Non-fiction literature,Plush toys,Poetry,Political thriller,Psychological thriller,Psychology,Puzzles,Romance,Science,Science fiction,Self-help,Short stories,Sports equipment,Suspense,Techno-thriller,Thriller,Toys,Travel
products,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Action,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.002225,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000568,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.002984,0.001158,0.000000,0.000000,0.000000
Adventure,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000569,0.000000,0.000000,0.000493,0.000000,0.000000,0.000586,0.000000,0.000000,0.000532,0.000000,0.000082,0.000000,0.000000,0.000000,0.000000,0.001330,0.000000,0.000000,0.000000,0.000000,0.001424,0.000000,0.000000,0.001161,0.000000,0.001377,0.000000,0.000000,0.000000,0.001197,0.001161,0.001286,0.000000,0.000000
Anthologies,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.001176,0.000000,0.000000,0.002264,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000585,0.000000,0.000000,0.000000,0.001121,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.002377,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Art supplies,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.002257,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.001115,0.000000,0.000000,0.000000,0.000000,0.000000
Biography,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000714,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000048,0.000000,0.000930,0.000000,0.000000,0.000000,0.000969,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000384,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000189
Books,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000147,0.000000
Building blocks,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000145,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.001159,0.000000,0.000000,0.000000,0.000000,0.000000,0.002800,0.000000,0.000000,0.000000,0.000000,0.000000
Business,0.000000,0.000000,0.000000,0.000000,0.000714,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.002023,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000997,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000049,0.000000,0.000000,0.000000,0.000000,

In [107]:
df.groupby('Product_Category')['products'].nunique()

Product_Category
Books          47
Clothing       70
Electronics    71
Grocery        60
Home Decor     70
Name: products, dtype: int64

In [116]:
from streamlit import file_uploader

In [126]:
import streamlit as st
import inspect

print(inspect.getsource(st.file_uploader))


    @gather_metrics("file_uploader")
    def file_uploader(
        self,
        label: str,
        type: str | Sequence[str] | None = None,
        accept_multiple_files: AcceptMultipleFiles = False,
        key: Key | None = None,
        help: str | None = None,
        on_change: WidgetCallback | None = None,
        args: WidgetArgs | None = None,
        kwargs: WidgetKwargs | None = None,
        *,  # keyword-only arguments:
        disabled: bool = False,
        label_visibility: LabelVisibility = "visible",
        width: WidthWithoutContent = "stretch",
    ) -> UploadedFile | list[UploadedFile] | None:
        r"""Display a file uploader widget.
        By default, uploaded files are limited to 200 MB each. You can
        configure this using the ``server.maxUploadSize`` config option. For
        more information on how to set config options, see |config.toml|_.

        .. |config.toml| replace:: ``config.toml``
        .. _config.toml: https://docs.streamlit.io/devel

In [124]:
dir(file_uploader.__class__.__class__)

['__abstractmethods__',
 '__annotations__',
 '__base__',
 '__bases__',
 '__basicsize__',
 '__call__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dictoffset__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__flags__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__instancecheck__',
 '__itemsize__',
 '__le__',
 '__lt__',
 '__module__',
 '__mro__',
 '__name__',
 '__ne__',
 '__new__',
 '__or__',
 '__prepare__',
 '__qualname__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__ror__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasscheck__',
 '__subclasses__',
 '__subclasshook__',
 '__text_signature__',
 '__weakrefoffset__',
 'mro']